#  프롬프트 엔지니어링 
- 효과적인 프롬프트 템플릿 설계

### **학습 목표:**  효과적인 프롬프트 템플릿의 기본 구조와 설계 원칙을 이해한다

---

# 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) LLM 설정`

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.3,
    top_p=0.9,
)

---
# **프롬프트 유형**

- 종류: 질문형, 지시형, 대화형, 조건부, 예시 기반 등 
- 이러한 프롬프트 유형들은 상황에 따라 조합하여 사용 가능
- 목적에 맞는 적절한 유형을 선택하는 것이 중요

`(1) 질문형 프롬프트 (Question Prompts)`
   - 정보 추출에 효과적
   - 구체적인 답변 유도 가능

In [5]:
from langchain.prompts import PromptTemplate
from langfuse import Langfuse, observe


# Langfuse 설정 (심리스 전환을 위한 구조)
LANGFUSE_ENABLED = os.getenv("LANGFUSE_ENABLED", "false").lower() == "true"

if LANGFUSE_ENABLED:
    langfuse = Langfuse(
        secret_key=os.getenv("LANGFUSE_SECRET_KEY"),
        public_key=os.getenv("LANGFUSE_PUBLIC_KEY"),
        host=os.getenv("LANGFUSE_HOST")
    )
    print("✅ Langfuse 추적이 활성화되었습니다.")
else:
    langfuse = None
    print("ℹ️ Langfuse 추적이 비활성화되었습니다.")

# 조건부 observe 데코레이터 (심리스 전환 핵심)
def conditional_observe(name=None, **kwargs):
    def decorator(func):
        if LANGFUSE_ENABLED and langfuse:
            return observe(name=name, **kwargs)(func)
        return func
    return decorator


# 단순 질문형 프롬프트
question_prompt = PromptTemplate(
    template="다음 주제에 대해 무엇을 알고 있나요?: {topic}",
    input_variables=["topic"]
)

# LCEL chain 구성
chain = question_prompt | llm

# Langfuse 추적이 적용된 질문 함수
@conditional_observe(name="quantum_computing_qa")
def ask_about_quantum_computing(topic: str):
    """양자 컴퓨팅에 대한 질문을 처리하는 함수"""
    output = chain.invoke({"topic": topic})
    return output

# 실행
if __name__ == "__main__":
    topic = "양자 컴퓨팅"
    output = ask_about_quantum_computing(topic)
    pprint(output.content)


✅ Langfuse 추적이 활성화되었습니다.
('양자 컴퓨팅에 대해 알고 있는 내용을 정리해 드리겠습니다.\n'
 '\n'
 '### 양자 컴퓨팅이란?\n'
 '양자 컴퓨팅(Quantum Computing)은 양자역학의 원리를 이용해 정보를 처리하는 컴퓨팅 기술입니다. 기존의 고전 컴퓨터가 '
 '비트(bit)를 사용해 0 또는 1의 값을 가지는 반면, 양자 컴퓨터는 양자 비트(큐비트, qubit)를 사용합니다. 큐비트는 0과 1의 '
 '상태를 동시에 가질 수 있는 중첩(superposition) 상태를 활용할 수 있어, 특정 계산에서 훨씬 빠른 처리 속도를 기대할 수 '
 '있습니다.\n'
 '\n'
 '### 주요 개념\n'
 '- **큐비트(Qubit)**: 양자 컴퓨터의 기본 단위로, 0과 1의 중첩 상태를 가질 수 있습니다.\n'
 '- **중첩(Superposition)**: 큐비트가 여러 상태를 동시에 가질 수 있는 현상.\n'
 '- **얽힘(Entanglement)**: 두 개 이상의 큐비트가 서로 강하게 연결되어, 한 큐비트의 상태가 다른 큐비트의 상태에 '
 '즉각적으로 영향을 미치는 현상.\n'
 '- **양자 게이트(Quantum Gate)**: 큐비트의 상태를 변화시키는 연산자. 고전 컴퓨터의 논리 게이트에 해당합니다.\n'
 '- **양자 알고리즘**: 양자 컴퓨터에서 실행되는 알고리즘으로, 대표적으로 쇼어 알고리즘(Shor’s algorithm, 소인수분해), '
 '그로버 알고리즘(Grover’s algorithm, 검색 문제)이 있습니다.\n'
 '\n'
 '### 양자 컴퓨팅의 장점\n'
 '- 특정 문제(예: 소인수분해, 최적화 문제, 시뮬레이션 등)에서 고전 컴퓨터보다 훨씬 빠른 계산 가능.\n'
 '- 복잡한 분자 및 재료 시뮬레이션에 유리하여 신약 개발, 신소재 연구에 활용 기대.\n'
 '\n'
 '### 현재 상황과 도전 과제\n'
 '- **하드웨어 개발**: 큐비트 수와 안정성, 오류율 개선이 필요합니다.\n'

In [6]:
### from_template 메소드를 사용한 방법

from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI
from pprint import pprint

# from_template을 사용한 PromptTemplate 생성
question_template = "다음 주제에 대해 무엇을 알고 있나요?: {topic}"
question_prompt = PromptTemplate.from_template(question_template)

# LCEL chain 구성
chain = question_prompt | llm

# 질문 실행
topic = "양자 컴퓨팅"
output = chain.invoke({"topic": topic})
pprint(output.content)

('양자 컴퓨팅(Quantum Computing)은 양자역학의 원리를 이용하여 정보를 처리하는 차세대 컴퓨팅 기술입니다. 기존의 고전 '
 '컴퓨터가 비트(bit)를 사용하여 0 또는 1의 값을 가지는 반면, 양자 컴퓨터는 양자 비트(큐비트, qubit)를 사용합니다. 큐비트는 '
 '0과 1 상태가 동시에 존재할 수 있는 중첩(superposition) 상태를 가질 수 있으며, 얽힘(entanglement)이라는 '
 '양자역학적 현상을 통해 여러 큐비트가 서로 강하게 연결될 수 있습니다.\n'
 '\n'
 '이러한 특성 덕분에 양자 컴퓨터는 특정 계산 문제에서 고전 컴퓨터보다 훨씬 빠른 속도로 문제를 해결할 수 있습니다. 예를 들어, '
 '소인수분해, 최적화 문제, 양자 시뮬레이션, 암호 해독 등에서 뛰어난 성능을 보일 것으로 기대됩니다.\n'
 '\n'
 '현재 양자 컴퓨팅은 아직 초기 단계에 있으며, 큐비트의 수와 안정성, 오류 수정 등의 기술적 도전 과제가 많습니다. 하지만 구글, '
 'IBM, 마이크로소프트, 인텔 등 여러 기업과 연구기관이 활발히 연구 중이며, 점차 실용적인 양자 컴퓨터 개발에 가까워지고 있습니다.')


**[참고]** `from_template` 방식의 장점:
- 코드가 더 간결해진다
- 입력 변수를 수동으로 지정할 필요가 없다
- 템플릿에서 사용된 변수를 자동으로 추출한다

In [7]:
# 분석 질문형 
analysis_prompt = PromptTemplate(
    template="""다음 텍스트에서 주요 논점은 무엇인가요? 세 가지로 설명해주세요.

    [텍스트]
    {text}

    [답변]
    """,
    input_variables=["text"]
)

# LCEL
chain = analysis_prompt | llm

# 질문
text = """양자 컴퓨팅은 양자역학의 원리를 이용하여 정보를 처리하는 컴퓨팅 기술이다. 
양자 컴퓨팅은 전통적인 컴퓨팅과는 다르게 양자역학의 원리를 이용하여 정보를 처리한다. 
양자 컴퓨팅은 양자역학의 원리를 이용하여 정보를 처리하는 컴퓨팅 기술이다.
"""

output = chain.invoke({"text": text})
pprint(output.content)

('[답변]  \n'
 '1. 양자 컴퓨팅은 양자역학의 원리를 기반으로 정보를 처리하는 기술이다.  \n'
 '2. 전통적인 컴퓨팅과는 다른 방식으로 작동한다는 점이 특징이다.  \n'
 '3. 양자 컴퓨팅의 핵심은 양자역학 원리를 활용한다는 것이다.')


### **[실습 1]**

- 앞의 예제를 from_template 메소드를 사용하는 방식으로 구현하세요.

In [11]:
analysis_prompt = PromptTemplate(
    template="""다음에 대해 초등학교 3학년이 이해할 수 있도록 쉽게 설명해주세요.

    [텍스트]
    {text}

    [답변]
    """,
    input_variables=["text"]
)

# LCEL
chain = analysis_prompt | llm

# 질문
text = """선형대수와 미분, 적분
"""

output = chain.invoke({"text": text})
pprint(output.content)

('안녕! 선형대수, 미분, 적분에 대해 쉽게 설명해줄게.\n'
 '\n'
 '1. **선형대수**  \n'
 '선형대수는 숫자나 그림을 이용해서 문제를 푸는 방법이야. 예를 들어, 여러 개의 숫자를 한꺼번에 계산하거나, 여러 가지 길이나 방향을 '
 '다룰 때 사용해. 마치 여러 개의 블록을 쌓아서 모양을 만드는 것처럼 생각하면 돼.\n'
 '\n'
 '2. **미분**  \n'
 "미분은 '변하는 속도'를 알아보는 거야. 예를 들어, 자동차가 얼마나 빨리 달리는지 알고 싶을 때 쓰는 방법이야. 자동차가 어느 순간에 "
 '얼마나 빨리 가는지 알려주는 거지.\n'
 '\n'
 '3. **적분**  \n'
 "적분은 '모든 작은 부분을 더해서 전체를 구하는 것'이야. 예를 들어, 작은 조각들을 모아서 큰 퍼즐을 완성하는 것처럼, 작은 부분들을 "
 '다 더해서 전체 크기나 양을 구하는 거야.\n'
 '\n'
 '쉽게 말하면, 선형대수는 숫자와 그림을 다루는 방법, 미분은 변하는 빠르기를 보는 방법, 적분은 작은 것들을 모아서 큰 것을 만드는 '
 '방법이야!')


`(2) 지시형 프롬프트 (Instruction Prompts)`
   - 명확한 작업 수행 지시
   - 단계별 처리 가능

In [12]:
from langchain_core.output_parsers import StrOutputParser
# 작업 지시형
task_prompt = PromptTemplate(
    template="다음 텍스트를 한국어로 번역하세요:\n\n[텍스트]\n{text}",
    input_variables=["text"]
)

# LCEL
chain = task_prompt | llm | StrOutputParser()

# 질문
text = "Quantum computing is a computing technology that uses the principles of quantum mechanics to process information."
output = chain.invoke({"text": text})
pprint(output)

'양자 컴퓨팅은 양자 역학의 원리를 이용하여 정보를 처리하는 컴퓨팅 기술입니다.'


In [13]:
# 단계별 지시형
step_prompt = PromptTemplate(
    template="""다음 텍스트에 대해서 작업을 순서대로 수행하세요:

    [텍스트]
    {text}

    [작업 순서]
    1. 텍스트를 1문장으로 요약
    2. 핵심 키워드 3개 추출
    3. 감정 분석 수행(긍정/부정/중립)

    [작업 결과]
    """,
    input_variables=["text"]
)

# LCEL
chain = step_prompt | llm | StrOutputParser()

# 질문
text = """
양자 컴퓨팅은 양자역학의 원리를 바탕으로 데이터를 처리하는 새로운 형태의 계산 방식이다. 
기존의 고전적 컴퓨터는 0과 1로 이루어진 이진법(bit)을 사용하여 데이터를 처리하지만, 
양자 컴퓨터는 양자 비트(큐비트, qubit)를 사용하여 훨씬 더 복잡하고 빠른 계산을 수행할 수 있다. 

큐비트는 동시에 0과 1의 상태를 가질 수 있는 양자 중첩(superposition) 상태를 활용하며, 
이를 통해 병렬 계산과 같은 고급 기능이 가능하다.
""" 

output = chain.invoke({"text": text})
pprint(output)

('[작업 결과]\n'
 '\n'
 '1. 요약: 양자 컴퓨팅은 양자역학의 원리를 이용해 큐비트를 활용함으로써 기존 컴퓨터보다 훨씬 빠르고 복잡한 계산을 가능하게 하는 새로운 '
 '계산 방식이다.  \n'
 '2. 핵심 키워드: 양자 컴퓨팅, 큐비트, 양자 중첩  \n'
 '3. 감정 분석: 중립')


### **[실습 2]**

- 두 개의 문장을 입력받아서, 두 문장의 맥락이 일치하는지 여부를 비교 분석하는 체인을 구성하세요.
- PromptTemplate를 사용하고, 두 문장을 입력받을 수 있도록 합니다. 
- 단계별 지시형 프롬프트로 작성합니다. 

In [19]:
# 단계별 지시형
step_prompt = PromptTemplate(
    template="""다음 두개의 문장에 대해서 맥락이 일치하는지 판단하세요:

    [a문장]
    {a_text}

    [b문장]
    {b_text}

    [작업 순서]
    1. 문장을 간단히 요약
    2. 요약의 핵심 키워드 3개 추출
    3. 키워드 기반으로 맥락 일치 여부 판단(일치/불일치)

    [작업 결과]
    """,
    input_variables=["b_text", "a_text"]
)

# LCEL
chain = step_prompt | llm | StrOutputParser()

# 질문
a_text = """
사람은 언어를 사용하여 의사소통을 한다.
""" 

b_text = """
인간은 언어를 통해 서로 소통한다.
"""
output = chain.invoke({"b_text": b_text, "a_text": a_text})
pprint(output)

('[작업 결과]\n'
 '\n'
 '1. 문장 요약  \n'
 '- a문장: 사람은 언어로 의사소통한다.  \n'
 '- b문장: 인간은 언어를 통해 서로 소통한다.\n'
 '\n'
 '2. 핵심 키워드 3개 추출  \n'
 '- a문장: 사람, 언어, 의사소통  \n'
 '- b문장: 인간, 언어, 소통\n'
 '\n'
 '3. 맥락 일치 여부 판단  \n'
 "- 두 문장 모두 '사람/인간'이 '언어'를 사용하여 '의사소통/소통' 한다는 의미로, 표현만 다를 뿐 내용과 맥락이 동일함.  \n"
 '- 따라서 맥락은 일치함.\n'
 '\n'
 '[최종 판단] 맥락 일치')


`(3) 대화형 프롬프트 (Conversational Prompts)`
   - 자연스러운 상호작용
   - 문맥 유지 가능

In [20]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# ChatPromptTemplate 객체를 직접 생성 (메시지 리스트, 입력 변수 리스트)
chat_prompt = ChatPromptTemplate(
    messages=[
        ("system", "당신은 친절한 고객 서비스 담당자입니다."),
        ("human", "{customer_message}")
    ],
    input_variables=["customer_message"]
)

# LCEL chain 구성
chain = chat_prompt | llm | StrOutputParser()

# 질문 실행
customer_message = "안녕하세요. 제품 배송 문제로 연락드렸어요."
output = chain.invoke({"customer_message": customer_message})
pprint(output)

'안녕하세요! 불편을 드려 죄송합니다. 배송 문제에 대해 자세히 말씀해 주시면 빠르게 도와드리겠습니다. 어떤 문제가 있으신가요?'


In [21]:
from langchain_core.prompts import ChatPromptTemplate

# from_messages 메소드를 사용한 ChatPromptTemplate 생성
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 고객 서비스 담당자입니다."),
    ("human", "{customer_message}"),
])

# LCEL
chain = chat_prompt | llm | StrOutputParser()

# 질문
customer_message = "안녕하세요. 제품 배송 문제로 연락드렸어요."
output = chain.invoke({"customer_message": customer_message})
pprint(output)

'안녕하세요! 불편을 드려 죄송합니다. 배송 문제에 대해 자세히 말씀해 주시면 빠르게 도와드리겠습니다. 어떤 문제가 있으신가요?'


In [22]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

# 메시지 템플릿 사용
chat_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "당신은 친절한 고객 서비스 담당자입니다."
    ),
    HumanMessagePromptTemplate.from_template(
        "{customer_message}"
    )
])

# LCEL
chain = chat_prompt | llm | StrOutputParser()

# 질문
customer_message = "안녕하세요. 제품 배송 문제로 연락드렸어요."
output = chain.invoke({"customer_message": customer_message})
pprint(output)


'안녕하세요! 불편을 드려 죄송합니다. 배송 문제에 대해 자세히 말씀해 주시면 신속히 도와드리겠습니다. 어떤 문제가 있으신가요?'


### **[실습 3]**

- 상품 리뷰를 분석하는 AI 체인을 대화형 프롬프트로 구성합니다. 
- 메시지 템플릿을 사용하여 메시지 역할을 구분합니다. 

In [23]:
# 예시 리뷰
reviews = [
    "이 블루투스 이어폰 정말 만족스러워요! 음질도 좋고 배터리도 오래가요. 다만 케이스가 좀 큰 감이 있네요.",
    "배송은 빨랐는데 제품 품질이 기대에 못 미쳐요. 터치감이 둔하고 연결이 자주 끊깁니다. 가성비 생각하면 그냥 쓸만하네요.",
    "가격대비 괜찮은 것 같아요. 디자인도 깔끔하고 기본적인 기능은 다 갖췄네요. 추천합니다!"
]

In [24]:
# 리뷰 분석 템플릿 생성
review_prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "당신은 상품 리뷰 분석 전문가입니다. 리뷰의 감정과 주요 포인트를 분석합니다."
        ),
    HumanMessagePromptTemplate.from_template(
        "다음 리뷰를 분석해주세요: {review}"
        )
])

# LCEL
review_chain = review_prompt | llm | StrOutputParser()# 여기에 코드를 작성하세요.


# 각 리뷰 분석 실행
for idx, review in enumerate(reviews, 1):
    print(f"\n=== 리뷰 {idx} 분석 결과 ===")
    result = review_chain.invoke({"review": review})
    print(result)


=== 리뷰 1 분석 결과 ===
리뷰 분석 결과:

감정: 긍정적  
주요 포인트:  
- 음질이 좋음  
- 배터리 지속 시간이 김  
- 케이스 크기가 다소 큼 (단점)  

종합적으로, 제품에 대해 매우 만족하며 음질과 배터리 성능을 특히 높이 평가하고 있으나, 케이스 크기에 대해서는 아쉬움을 표현하고 있습니다.

=== 리뷰 2 분석 결과 ===
리뷰 분석 결과:

감정: 전반적으로 부정적이지만, 가성비 측면에서는 약간 긍정적인 평가가 혼재되어 있습니다.

주요 포인트:
1. 배송 속도: 빠름 (긍정적)
2. 제품 품질: 기대 이하 (부정적)
3. 터치감: 둔함 (부정적)
4. 연결 상태: 자주 끊김 (부정적)
5. 가성비: 가격 대비 사용 가능함 (긍정적)

요약: 배송은 만족스러우나, 제품의 터치감과 연결 품질이 떨어져 아쉬움이 크다. 다만 가격 대비 성능은 무난한 편으로 평가됨.

=== 리뷰 3 분석 결과 ===
리뷰 분석 결과:

감정: 긍정적  
주요 포인트:  
- 가격 대비 만족스러운 품질  
- 깔끔한 디자인  
- 기본 기능 충실  
- 추천 의사 표현  

종합적으로, 사용자는 제품의 가성비와 디자인, 기능에 만족하며 긍정적으로 평가하고 있습니다.


`(4) 예시 기반 프롬프트 (Few-Shot Prompts)`
   - 원하는 출력 형식 명확화
   - 모델의 이해도 향상

In [25]:
few_shot_prompt = PromptTemplate(
    template="""다음은 텍스트를 요약하는 예시입니다:

원문: {example_input}
요약: {example_output}

이제 다음 텍스트를 같은 방식으로 50자 이내로 요약해주세요:
원문: {input_text}
요약:
""",
    input_variables=["example_input", "example_output", "input_text"]
)

# LCEL
chain = few_shot_prompt | llm | StrOutputParser()

# 예시 텍스트 
example_input = """인공지능(AI)은 인간의 학습능력, 추론능력, 지각능력, 자연언어의 이해능력 등을 
컴퓨터 프로그램으로 실현한 기술이다. 인공지능은 딥러닝, 기계학습 등 다양한 기술을 포함하며, 
최근에는 자율주행, 의료진단, 언어번역 등 다양한 분야에서 활용되고 있다."""

example_output = "인공지능: 인간의 학습능력, 추론능력, 지각능력 등을 컴퓨터 프로그램으로 실현한 기술"

# 입력 텍스트 
input_text = """양자 컴퓨팅은 양자역학의 원리를 활용하여 정보를 처리하는 혁신적인 컴퓨팅 기술이다. 
기존의 디지털 컴퓨터가 0과 1의 이진법을 사용하는 것과 달리, 양자 컴퓨터는 중첩 상태를 활용하여 
동시에 여러 계산을 수행할 수 있다. 이러한 특성으로 인해 특정 문제에서는 기존 컴퓨터보다 
월등히 빠른 처리 속도를 보여준다. 현재는 아직 초기 단계지만, 암호화, 신약 개발, 기후 모델링 등 
다양한 분야에서 혁신적인 발전이 기대된다."""

# 체인 실행
output = chain.invoke({
    "example_input": example_input, 
    "example_output": example_output, 
    "input_text": input_text
})
pprint(output)

'요약: 양자 컴퓨팅: 양자역학 원리로 정보를 처리하는 혁신적 컴퓨팅 기술'


### **[실습 4]**

- [실습 3]의 상품 리뷰 분석 시스템의 입출력 형식을 예시로 추가해서 프롬프트를 작성합니다.
- 체인을 실해하여 테스트합니다. 

In [26]:
# 여기에 코드를 작성하세요.
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Few-Shot 예시를 포함한 시스템 메시지
system_message = """당신은 상품 리뷰 분석 전문가입니다. 리뷰를 분석하여 아래 형식으로 출력합니다:

예시 1:
리뷰: "이 노트북 정말 가볍고 좋아요! 배터리도 오래가고 화면도 선명해요. 다만 가격이 조금 비싸네요."
분석:
- 감정: 긍정적
- 장점: 가벼움, 배터리 수명, 화면 품질
- 단점: 높은 가격
- 종합: 전반적으로 만족, 가격 대비 가치 고려 필요

예시 2:
리뷰: "배송이 늦어져서 실망했어요. 제품 자체는 나쁘지 않은데 AS가 불편해요."
분석:
- 감정: 부정적
- 장점: 제품 품질 양호
- 단점: 늦은 배송, AS 서비스 불편
- 종합: 서비스 개선 필요
"""

# 프롬프트 템플릿 구성
review_prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=system_message),
    HumanMessagePromptTemplate.from_template("다음 리뷰를 분석해주세요:\n리뷰: {review}")
])

# 체인 구성
review_chain = review_prompt | llm | StrOutputParser()



# 테스트
test_review = "이 무선 마우스 디자인이 예쁘고 작동도 부드러워요. 근데 가끔 연결이 끊기는게 아쉽네요. 전체적으로는 만족합니다."
result = review_chain.invoke({"review": test_review})
print(result)

분석:
- 감정: 긍정적
- 장점: 예쁜 디자인, 부드러운 작동, 전반적인 만족도
- 단점: 가끔 연결 끊김 현상
- 종합: 디자인과 사용감에 만족하지만 연결 안정성 개선 필요


`(5) 조건부 프롬프트 (Conditional Prompts)`
   - 상황별 다른 처리
   - 유연한 응답 생성

In [27]:
# 조건부 프롬프트 템플릿 정의 (입력 텍스트에 따라 작업 유형을 지정)
conditional_prompt = PromptTemplate(
    template="""입력 텍스트: {text}

주어진 텍스트가 질문인 경우: 명확한 답변을 제공
주어진 텍스트가 진술문인 경우: 진술문의 사실 여부를 검증
주어진 텍스트가 요청사항인 경우: 수행 방법을 단계별로 설명

응답은 다음 형식을 따라주세요:
유형: [질문/진술문/요청사항]
내용: [상세 응답]""",
    input_variables=["text"]
)

# LCEL 체인 구성
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
chain = conditional_prompt | llm | StrOutputParser()

C:\Users\kaydash\AppData\Local\Temp\ipykernel_32620\497816025.py:16: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)


In [28]:
# 질문형 테스트
response = chain.invoke({
    "text": "인공지능은 인간의 일자리를 모두 대체하게 될까요?",
})
print(response)

유형: 질문  
내용: 인공지능이 인간의 일자리를 모두 대체할 가능성은 매우 낮습니다. 인공지능과 자동화 기술은 일부 직업과 업무를 대체하거나 보조할 수 있지만, 창의성, 감정적 지능, 복잡한 의사결정 등 인간 고유의 능력이 요구되는 분야에서는 여전히 인간의 역할이 중요합니다. 또한, 새로운 기술 발전에 따라 새로운 일자리도 창출될 가능성이 큽니다. 따라서 인공지능은 일자리의 형태와 업무 방식을 변화시키겠지만, 모든 일자리를 완전히 대체하지는 않을 것으로 예상됩니다.


In [29]:
# 진술문 테스트
response = chain.invoke({
    "text": "양자컴퓨터는 현재 모든 암호화 시스템을 무력화할 수 있다."
})
print(response)

유형: 진술문  
내용: "양자컴퓨터는 현재 모든 암호화 시스템을 무력화할 수 있다"는 진술은 사실이 아닙니다. 현재 상용화된 양자컴퓨터는 매우 제한된 규모이며, 기존의 대부분 암호화 시스템을 무력화할 만큼의 성능을 갖추지 못했습니다. 다만, 이론적으로 충분히 발전한 양자컴퓨터는 일부 암호화 알고리즘(예: RSA, ECC)을 깨뜨릴 수 있는 잠재력을 가지고 있으나, 이를 대비한 양자 내성 암호화 기술도 활발히 연구되고 있습니다. 따라서 현재 시점에서는 모든 암호화 시스템이 무력화되었다고 보기 어렵습니다.


In [30]:
# 요청사항 테스트
response = chain.invoke({
        "text": "파이썬으로 간단한 웹 스크래퍼를 만들고 싶습니다."
})
print(response)

유형: 요청사항  
내용: 파이썬으로 간단한 웹 스크래퍼를 만드는 방법을 단계별로 설명드리겠습니다.

1. 필요한 라이브러리 설치  
   - `requests`: 웹 페이지의 HTML을 가져오기 위해 사용  
   - `BeautifulSoup`: HTML을 파싱하여 원하는 데이터를 추출하기 위해 사용  
   ```bash
   pip install requests beautifulsoup4
   ```

2. 웹 페이지 요청 및 HTML 가져오기  
   ```python
   import requests

   url = 'https://example.com'  # 스크래핑할 웹 페이지 URL
   response = requests.get(url)
   html = response.text
   ```

3. HTML 파싱 및 데이터 추출  
   ```python
   from bs4 import BeautifulSoup

   soup = BeautifulSoup(html, 'html.parser')
   # 예: 모든 <a> 태그의 텍스트와 링크 추출
   for a_tag in soup.find_all('a'):
       print(a_tag.text, a_tag.get('href'))
   ```

4. 결과 활용  
   - 추출한 데이터를 파일로 저장하거나, 원하는 형태로 가공할 수 있습니다.

5. 주의사항  
   - 웹사이트의 robots.txt 파일을 확인하여 스크래핑이 허용되는지 확인하세요.  
   - 과도한 요청은 서버에 부담을 줄 수 있으니 적절한 딜레이를 두세요.

이 과정을 통해 간단한 웹 스크래퍼를 만들 수 있습니다. 필요에 따라 더 복잡한 기능도 추가 가능합니다.


--- 

# **프롬프트 엔지니어링 개념**

1. **개념**:
    - AI 모델에게 효과적인 지시를 제공하여 원하는 결과를 얻어내는 기술
    - 입력(프롬프트)을 최적화하여 출력의 품질을 향상하는 방법 

2. **원칙**:

    1. **명확성(Clarity)**
        - 모호하지 않은 명확한 지시사항 제공
        - 구체적인 요구사항과 제약조건 명시
        - 예시: "5개의 짧은 문장으로 요약해주세요" vs "요약해주세요"

    1. **맥락성(Context)**
        - 관련 배경 정보 제공
        - 목적과 의도 명시
        - 대상 독자나 사용 환경 설명

    1. **구조화(Structure)**
        - 체계적인 형식 사용
        - 단계별 지시사항 제공
        - 원하는 출력 형식 명시

---
## 1. **명확성(Clarity)**

- **프롬프트의 명확성**은 AI 모델과의 효과적인 소통을 위한 핵심 요소

- 불필요한 내용을 제외하고 **핵심 요구사항**에만 집중하여 작성

- 원하는 결과물에 대해 **구체적이고 정확한 지시**를 제공

In [31]:
from langchain_core.prompts import PromptTemplate

# 명확한 지시사항이 포함된 프롬프트 템플릿
clear_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""
    주제: {topic}
    
    다음 기준을 반드시 준수하여 설명하시오:
    1. 정확히 3문장으로 작성할 것 (Bullet point 사용하여 구분)
    2. 각 문장은 20단어 이내로 작성할 것
    3. 전문 용어는 괄호 안에 간단한 설명을 포함할 것
    """
)

# LCEL 체인
clear_chain = clear_prompt | llm 

# 테스트
result = clear_chain.invoke({"topic": "인공지능"})
pprint(result.content)

('- 인공지능은 컴퓨터가 인간처럼 학습하고 문제를 해결하는 기술을 의미한다.  \n'
 '- 머신러닝(데이터로부터 학습하는 알고리즘)은 인공지능의 핵심 기술 중 하나이다.  \n'
 '- 인공지능은 의료, 금융, 자율주행 등 다양한 분야에서 혁신을 이끌고 있다.')


---
## 2. **맥락성(Context)**

- **맥락 제공**은 AI가 작업의 배경과 목적을 이해하는데 필수적인 요소

- 프롬프트에 **배경 정보**, **목적**, **대상 환경**을 명확히 포함

- 적절한 맥락 제공은 AI의 **출력 품질**과 **정확도**를 크게 향상

In [32]:
from langchain_core.prompts import ChatPromptTemplate

# 맥락이 부족한 프롬프트의 예
bad_prompt = ChatPromptTemplate.from_template("""
당신은 친절한 AI 어시스턴트입니다. 사용자의 질문에 답변해주세요.
사용자 질문: {user_question}                                              
""")

# 맥락이 풍부한 프롬프트의 예
good_prompt = ChatPromptTemplate.from_template("""
사용자 질문: {user_question}

배경: 65세 이상 노인을 대상으로 하는 스마트폰 교육 프로그램을 진행하고 있습니다.
목적: 처음 스마트폰을 사용하는 노인들이 기본 기능을 쉽게 익힐 수 있도록 돕고자 합니다.
대상: 디지털 기기 사용 경험이 거의 없는 노인입니다.

위 맥락을 고려하여 응답해주세요.

응답 형식:
- 쉬운 용어 사용
- 단계별 설명
- 구체적인 예시 포함
""")


# LCEL 체인
good_chain = good_prompt | llm 
bad_chain = bad_prompt | llm

# 테스트
question = "아이폰에 카카오톡 설치하는 방법을 알려주세요."
good_result = good_chain.invoke({"user_question": question})
bad_result = bad_chain.invoke({"user_question": question})

print("맥락이 부족한 프롬프트의 결과")
pprint(bad_result.content)
print("-"*100)

print("맥락이 풍부한 프롬프트의 결과")
pprint(good_result.content)

맥락이 부족한 프롬프트의 결과
('아이폰에 카카오톡을 설치하는 방법을 안내해드릴게요.\n'
 '\n'
 '1. **앱스토어 열기**  \n'
 '   아이폰에서 홈 화면에 있는 **App Store** 아이콘을 탭하여 앱스토어를 엽니다.\n'
 '\n'
 '2. **검색하기**  \n'
 '   하단의 돋보기 모양 아이콘(검색)을 탭한 후, 검색창에 **"카카오톡"**을 입력하고 검색합니다.\n'
 '\n'
 '3. **카카오톡 선택**  \n'
 '   검색 결과에서 **카카오톡(KakaoTalk)** 앱을 찾아 탭합니다. 개발사는 "Kakao Corp."입니다.\n'
 '\n'
 '4. **앱 설치하기**  \n'
 '   **받기** 또는 **클라우드 모양 아이콘**을 탭하여 앱을 다운로드하고 설치합니다. Apple ID 비밀번호를 묻거나 Face '
 'ID/Touch ID 인증을 요청할 수 있습니다.\n'
 '\n'
 '5. **설치 완료 후 열기**  \n'
 '   설치가 완료되면 **열기** 버튼을 눌러 카카오톡을 실행합니다.\n'
 '\n'
 '6. **회원가입 또는 로그인**  \n'
 '   기존 계정이 있으면 로그인하고, 없으면 회원가입 절차를 따라 계정을 만드시면 됩니다.\n'
 '\n'
 '필요하시면 추가로 카카오톡 사용법도 알려드릴 수 있습니다!')
----------------------------------------------------------------------------------------------------
맥락이 풍부한 프롬프트의 결과
('안녕하세요! 아이폰에 카카오톡을 설치하는 방법을 아주 쉽게 알려드릴게요. 천천히 따라 하시면 누구나 할 수 있어요.\n'
 '\n'
 '---\n'
 '\n'
 '### 1단계: 아이폰에서 ‘앱스토어’ 찾기  \n'
 '- 아이폰 바탕화면에서 **파란색 바탕에 흰색 글자 ‘App Store’** 아이콘을 찾아서 손가락으로 한 번 눌러주세요.  \n'
 '- 예를 들어, 집에서 

---
## 3. **구조화(Structure)**

- 프롬프트 입력의 구조화: PromptTemplate, ChatPromptTemplate 사용
- LLM 출력의 구조화: OutputParser, Schema 사용

- 기대효과:
    - 일관된 형식의 입출력 보장
    - 데이터 처리 및 후속 작업의 용이성
    - 오류 처리의 체계화
    - 재사용성 향상

### 1) 프롬프트 템플릿 (Prompt Template)

- **프롬프트 템플릿**은 LLM과의 상호작용을 구조화하는 핵심 도구

- 템플릿은 다양한 입력값으로 **재사용**이 가능하며 **유연한 변수 처리**를 지원

- **검증 기능**과 **부분 포맷팅**을 통해 프롬프트 작성의 안정성을 보장

`(1) 기본 템플릿`



In [33]:
from langchain_core.prompts import PromptTemplate

# 템플릿 정의
template = """
다음 주제에 대해 설명해주세요: {topic}
포함해야 할 내용: {content}
글자수: {length}
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["topic", "content", "length"]
)

# 템플릿 사용
formatted_prompt = prompt.format(
    topic="인공지능",
    content="정의, 역사, 응용분야",
    length="500자"
)

print(formatted_prompt)


다음 주제에 대해 설명해주세요: 인공지능
포함해야 할 내용: 정의, 역사, 응용분야
글자수: 500자



`(2) from_template 메소드`

In [34]:
from langchain_core.prompts import PromptTemplate

# 템플릿 문자열 정의
template = """
다음 주제에 대해 설명해주세요: {topic}
포함해야 할 내용: {content}
글자수: {length}
"""

# 템플릿 생성 (템플릿 문자열 지정)
prompt = PromptTemplate.from_template(template)

# 템플릿 사용 (입력 변수 지정)
formatted_prompt = prompt.format(
    topic="인공지능",
    content="정의, 역사, 응용분야",
    length="500자"
)

print(formatted_prompt)


다음 주제에 대해 설명해주세요: 인공지능
포함해야 할 내용: 정의, 역사, 응용분야
글자수: 500자



`(3) 템플릿 검증 및 부분 포맷팅`

In [35]:
from langchain_core.prompts import PromptTemplate

# 템플릿 생성 
template = PromptTemplate(
    template="{product}의 다음 특징을 분석해주세요: {feature}",  # 템플릿 문자열
    input_variables=["product", "feature"],  # 입력 변수 목록
    validate_template=True  # 템플릿 유효성 검증
)

# 부분적으로 변수 채우기
partial_prompt = template.partial(product="스마트폰")

print(f"부분적으로 변수 채워진 템플릿: {partial_prompt}")
print("-"*100)

# 나중에 나머지 변수 채우기
final_prompt1 = partial_prompt.format(feature="카메라")

print(f"나머지 변수 채워진 템플릿: {final_prompt1}")
print("-"*100)

# 다른 특징을 분석하도록 변수 변경
final_prompt2 = partial_prompt.format(feature="배터리 수명")

print(f"다른 변수 채워진 템플릿: {final_prompt2}")
print("-"*100)

# 모든 변수를 한 번에 채우기
final_prompt3 = template.format(product="노트북", feature="배터리 수명")

print(f"모든 변수 채워진 템플릿: {final_prompt3}")

부분적으로 변수 채워진 템플릿: input_variables=['feature'] input_types={} partial_variables={'product': '스마트폰'} template='{product}의 다음 특징을 분석해주세요: {feature}' validate_template=True
----------------------------------------------------------------------------------------------------
나머지 변수 채워진 템플릿: 스마트폰의 다음 특징을 분석해주세요: 카메라
----------------------------------------------------------------------------------------------------
다른 변수 채워진 템플릿: 스마트폰의 다음 특징을 분석해주세요: 배터리 수명
----------------------------------------------------------------------------------------------------
모든 변수 채워진 템플릿: 노트북의 다음 특징을 분석해주세요: 배터리 수명


In [36]:
# 템플릿 유효성 검증 실패
try:
    invalid_prompt = template.format(product="스마트폰")
except ValueError as e:
    print(f"템플릿 유효성 검증 실패: {e}")

KeyError: 'feature'

### 2) 챗 프롬프트 템플릿 (Chat Prompt Template)

- **챗 프롬프트 템플릿**은 대화형 AI와의 상호작용을 위한 특화된 템플릿

- 시스템/사용자/어시스턴트 등 **다양한 역할**의 메시지를 구조화

- 대화의 **맥락과 흐름**을 유지하면서 일관된 상호작용 가능

`(1) 메시지 템플릿 사용`

In [ ]:
### 2. 복잡한 템플릿 구성:

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 개별 메시지 템플릿 생성
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 {role} 전문가입니다. {style} 스타일로 답변해주세요."
)

human_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)

# from_messages 메소드 사용 (여러 개의 메시지들을 원소로 갖는 리스트로 구성)
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    human_message
])

# 템플릿 사용
formatted_prompt = chat_prompt.format(
    role="인공지능",
    style="친절한",
    question="인공지능의 정의를 설명해주세요."
)

print(formatted_prompt)

`(2) 문자열 템플릿 사용`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 시스템 메시지와 사용자 메시지를 포함한 템플릿 정의
template = """
당신은 {role} 전문가입니다. {style} 스타일로 답변해주세요.

{question}
"""

# from_template 메소드 사용 (단일 템플릿 문자열 직접 사용)
chat_prompt = ChatPromptTemplate.from_template(template)

# 템플릿 사용
formatted_prompt = chat_prompt.format(
    role="인공지능",
    style="친절한",
    question="인공지능의 정의를 설명해주세요."
)

# ChatPromptTemplate을 직접 생성 - 이때는 HumanMessage로 처리됨 (SystemMessage는 별도로 추가해야 함)
print(formatted_prompt)

### 2)  OutputParser 활용

- **OutputParser**는 LLM의 출력을 다양한 데이터 형식으로 변환하는 도구

- **문자열**, **JSON**, **XML** 등 여러 형식의 파싱을 지원

- 파싱된 출력은 **다른 시스템**이나 **프로세스**와 연동하는데 유용

`(1) JSONOutputParser`
- LLM의 출력을 **구조화된 JSON**으로 변환
- 파서는 출력의 **데이터 유효성**을 검증하고 일관된 형식을 보장

In [37]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field
from typing import List

# 관광지 정보를 위한 Pydantic 모델 정의 (이름, 위치, 카테고리, 주요 관람 포인트)
class TouristSpot(BaseModel):
    name: str = Field(description="관광명소 이름")
    location: str = Field(description="위치 (구/동 정보)")
    category: str = Field(description="카테고리 (궁궐/박물관/쇼핑 등)")
    highlights: List[str] = Field(description="주요 관람 포인트")

# JsonOutputParser 파서 설정 (Pydantic 모델 지정)
parser = JsonOutputParser(pydantic_object=TouristSpot)

# parser의 get_format_instructions 메소드 사용 (포맷 지시사항 출력)
pprint(parser.get_format_instructions())

('The output should be formatted as a JSON instance that conforms to the JSON '
 'schema below.\n'
 '\n'
 'As an example, for the schema {"properties": {"foo": {"title": "Foo", '
 '"description": "a list of strings", "type": "array", "items": {"type": '
 '"string"}}}, "required": ["foo"]}\n'
 'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the '
 'schema. The object {"properties": {"foo": ["bar", "baz"]}} is not '
 'well-formatted.\n'
 '\n'
 'Here is the output schema:\n'
 '```\n'
 '{"properties": {"name": {"description": "관광명소 이름", "title": "Name", "type": '
 '"string"}, "location": {"description": "위치 (구/동 정보)", "title": "Location", '
 '"type": "string"}, "category": {"description": "카테고리 (궁궐/박물관/쇼핑 등)", '
 '"title": "Category", "type": "string"}, "highlights": {"description": "주요 관람 '
 '포인트", "items": {"type": "string"}, "title": "Highlights", "type": "array"}}, '
 '"required": ["name", "location", "category", "highlights"]}\n'
 '```')


In [38]:
# 프롬프트 템플릿 정의 (parser의 get_format_instructions 메소드 사용)
prompt = PromptTemplate(
    template="""서울의 다음 관광명소에 대한 상세 정보를 제공해주세요.
{format_instructions}

관광지: {spot_name}
""",
    input_variables=["spot_name"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 완성된 프롬프트 템플릿 출력 (포맷 지시사항 포함)
pprint(prompt.format(spot_name="경복궁"))

('서울의 다음 관광명소에 대한 상세 정보를 제공해주세요.\n'
 'The output should be formatted as a JSON instance that conforms to the JSON '
 'schema below.\n'
 '\n'
 'As an example, for the schema {"properties": {"foo": {"title": "Foo", '
 '"description": "a list of strings", "type": "array", "items": {"type": '
 '"string"}}}, "required": ["foo"]}\n'
 'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the '
 'schema. The object {"properties": {"foo": ["bar", "baz"]}} is not '
 'well-formatted.\n'
 '\n'
 'Here is the output schema:\n'
 '```\n'
 '{"properties": {"name": {"description": "관광명소 이름", "title": "Name", "type": '
 '"string"}, "location": {"description": "위치 (구/동 정보)", "title": "Location", '
 '"type": "string"}, "category": {"description": "카테고리 (궁궐/박물관/쇼핑 등)", '
 '"title": "Category", "type": "string"}, "highlights": {"description": "주요 관람 '
 '포인트", "items": {"type": "string"}, "title": "Highlights", "type": "array"}}, '
 '"required": ["name", "location", "category", "highlights"]}\n'

In [39]:
# 체인 구성
chain = prompt | llm | parser

# 실행 예시
result = chain.invoke({
    "spot_name": "경복궁"
})

# 결과 출력
pprint(result)

{'category': '궁궐',
 'highlights': ['조선 시대의 대표적인 궁궐로서 한국 전통 건축 양식을 감상할 수 있음',
                '근정전, 경회루 등 주요 건물과 아름다운 정원',
                '한복 체험 및 수문장 교대식 관람 가능',
                '국립고궁박물관과 국립민속박물관이 인접해 있어 역사와 문화를 함께 즐길 수 있음'],
 'location': '종로구 세종로',
 'name': '경복궁'}


`(2) XMLOutputParser`
- LLM의 출력을 **구조화된 XML** 형식으로 변환
- XML 구조는 **계층적 데이터**를 표현하는데 효과적
- 패키지 설치: defusedxml 

In [40]:
from langchain_core.output_parsers import XMLOutputParser

# XML 파서 설정 - 원하는 태그 구조 정의
parser = XMLOutputParser(tags=["tourist_spot", "name", "location", "category", "highlights", "point"])

# parser의 get_format_instructions 메소드 사용 (포맷 지시사항 출력)
pprint(parser.get_format_instructions())

('The output should be formatted as a XML file.\n'
 '1. Output should conform to the tags below.\n'
 '2. If tags are not given, make them on your own.\n'
 '3. Remember to always open and close all the tags.\n'
 '\n'
 'As an example, for the tags ["foo", "bar", "baz"]:\n'
 '1. String "<foo>\n'
 '   <bar>\n'
 '      <baz></baz>\n'
 '   </bar>\n'
 '</foo>" is a well-formatted instance of the schema.\n'
 '2. String "<foo>\n'
 '   <bar>\n'
 '   </foo>" is a badly-formatted instance.\n'
 '3. String "<foo>\n'
 '   <tag>\n'
 '   </tag>\n'
 '</foo>" is a badly-formatted instance.\n'
 '\n'
 'Here are the output tags:\n'
 '```\n'
 "['tourist_spot', 'name', 'location', 'category', 'highlights', 'point']\n"
 '```')


In [41]:
# 프롬프트 템플릿 정의
prompt = PromptTemplate(
    template="""서울의 다음 관광명소에 대한 상세 정보를 XML 형식으로 제공해주세요.
{format_instructions}

관광지: {spot_name}""",
    input_variables=["spot_name"],
    partial_variables={"format_instructions": parser.get_format_instructions()}  
)

# 완성된 프롬프트 템플릿 출력 (포맷 지시사항 포함)
pprint(prompt.format(spot_name="경복궁"))

('서울의 다음 관광명소에 대한 상세 정보를 XML 형식으로 제공해주세요.\n'
 'The output should be formatted as a XML file.\n'
 '1. Output should conform to the tags below.\n'
 '2. If tags are not given, make them on your own.\n'
 '3. Remember to always open and close all the tags.\n'
 '\n'
 'As an example, for the tags ["foo", "bar", "baz"]:\n'
 '1. String "<foo>\n'
 '   <bar>\n'
 '      <baz></baz>\n'
 '   </bar>\n'
 '</foo>" is a well-formatted instance of the schema.\n'
 '2. String "<foo>\n'
 '   <bar>\n'
 '   </foo>" is a badly-formatted instance.\n'
 '3. String "<foo>\n'
 '   <tag>\n'
 '   </tag>\n'
 '</foo>" is a badly-formatted instance.\n'
 '\n'
 'Here are the output tags:\n'
 '```\n'
 "['tourist_spot', 'name', 'location', 'category', 'highlights', 'point']\n"
 '```\n'
 '\n'
 '관광지: 경복궁')


In [42]:
# 체인 구성
chain = prompt | llm | parser

# 실행 예시
result = chain.invoke({
    "spot_name": "경복궁"
})

# 결과는 Python 딕셔너리 형태로 파싱되어 반환됨 
pprint(result)

{'tourist_spot': [{'name': '경복궁'},
                  {'location': '서울특별시 종로구 사직로 161'},
                  {'category': '역사/문화유적'},
                  {'highlights': [{'point': '조선 왕조의 정궁으로서 한국 전통 건축의 아름다움을 감상할 '
                                            '수 있음'},
                                  {'point': '근정전, 경회루 등 주요 건축물과 왕실 정원 관람 가능'},
                                  {'point': '한복 체험 및 전통 문화 행사 참여 기회 제공'},
                                  {'point': '경복궁 야간 개장 시 아름다운 야경 감상 가능'}]}]}


In [43]:
# XML 형식의 결과를 다시 파싱하여 출력 (가장 상위 태그를 제외한 내용만 출력)
result['tourist_spot']

[{'name': '경복궁'},
 {'location': '서울특별시 종로구 사직로 161'},
 {'category': '역사/문화유적'},
 {'highlights': [{'point': '조선 왕조의 정궁으로서 한국 전통 건축의 아름다움을 감상할 수 있음'},
   {'point': '근정전, 경회루 등 주요 건축물과 왕실 정원 관람 가능'},
   {'point': '한복 체험 및 전통 문화 행사 참여 기회 제공'},
   {'point': '경복궁 야간 개장 시 아름다운 야경 감상 가능'}]}]

`(3) 사용자 정의(Custom) OutputParser`
- **사용자 정의 파서**는 **RunnableLambda**나 **RunnableGenerator**를 활용하여 구현
- 특정 비즈니스 요구사항에 맞는 **맞춤형 출력 형식**을 정의할 수 있음 
- 복잡한 **데이터 변환**과 **후처리 로직**을 유연하게 구현할 수 있음 

In [44]:
from langchain_core.messages import AIMessage
from langchain_core.prompts import PromptTemplate
from typing import Dict

# 사용자 정의 파서
def parse_tourist_spot(ai_message: AIMessage) -> Dict:
    """관광지 정보를 딕셔너리로 파싱"""
    lines = ai_message.content.split('\n')
    return {
        "name": lines[0] if lines else "",
        "description": ' '.join(lines[1:]) if len(lines) > 1 else ""
    }

# 프롬프트 템플릿
prompt = PromptTemplate(
    template="다음 관광지에 대해 첫 줄에 이름, 다음 줄부터 설명을 100자 내외로 작성해주세요:\n{spot_name}",
    input_variables=["spot_name"]
)

# 체인 구성 (invoke 방식)
invoke_chain = prompt | llm | parse_tourist_spot

# 체인 실행
result = invoke_chain.invoke({
    "spot_name": "경복궁"
})

pprint(result)

{'description': '조선 시대의 대표적인 궁궐로, 서울에 위치해 있습니다. 아름다운 건축과 역사적 의미가 돋보이는 관광 '
                '명소입니다.',
 'name': '경복궁  '}


In [45]:
## 위 구현을 stream 방식으로 변경

from langchain_core.runnables import RunnableGenerator
from langchain_core.messages import AIMessageChunk
from typing import Iterable
import time

# 사용자 정의 파서 (stream 방식) 
def streaming_parse_tourist_spot(chunks: Iterable[AIMessageChunk]) -> Iterable[str]:
    """실시간으로 관광지 정보 파싱"""
    for chunk in chunks:
        yield f"🏛 {chunk.content}"   # 🏛 이모지 추가 

# stream 방식 체인 구성 
streaming_chain = prompt | llm | RunnableGenerator(streaming_parse_tourist_spot)

# 체인 실행 
for chunk in streaming_chain.stream({"spot_name": "광화문"}):
    print(chunk)
    time.sleep(0.1)   # 시간 지연 추가 

🏛 
🏛 광
🏛 화
🏛 문
🏛   

🏛 서울
🏛 의
🏛  중심
🏛 에
🏛  위치
🏛 한
🏛  조
🏛 선
🏛  시대
🏛 의
🏛  정
🏛 문
🏛 으로
🏛 ,
🏛  역사
🏛 적
🏛  상
🏛 징
🏛 성과
🏛  문화
🏛 적
🏛  의미
🏛 가
🏛  깊
🏛 은
🏛  명
🏛 소
🏛 입니다
🏛 .
🏛 


**[참고] 이모지(emoji) 입력 방법**

1. Windows에서:
    - `Windows 키 + .` (윈도우 키와 마침표를 동시에 누름)
    - 또는 `Windows 키 + ;` (윈도우 키와 세미콜론을 동시에 누름)

2. Mac에서:
    - `fn(지구본) 키 + E` 

`(3) 구조화된 출력 프롬프트 (Structured Output Prompts)`
   - 일관된 형식의 응답
   - 데이터 처리 용이

In [48]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List, Literal
from pprint import pprint

# LLM 설정 (기존과 동일)
llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.3,
    top_p=0.9,
)

# Pydantic 모델로 변경 (TypedDict 대신 BaseModel 사용)
class AnalysisResult(BaseModel):
    """텍스트 분석 결과를 담는 스키마"""
    summary: str = Field(..., description="텍스트의 핵심 내용 요약")
    keywords: List[str] = Field(..., description="텍스트에서 추출한 주요 키워드")
    sentiment: Literal["긍정", "부정", "중립"] = Field(..., description="텍스트의 전반적인 감정 분석 결과")

# JsonOutputParser 설정 (Pydantic 모델 지정)
parser = JsonOutputParser(pydantic_object=AnalysisResult)

# 단계별 지시형 프롬프트 (기존과 동일)
step_prompt = PromptTemplate(
    template="""다음 텍스트에 대해서 작업을 순서대로 수행하세요:

    [텍스트]
    {text}

    [작업 순서]
    1. 텍스트를 1문장으로 요약
    2. 핵심 키워드 3개 추출
    3. 감정 분석 수행(긍정/부정/중립)

    [작업 결과]
    {format_instructions}
    """,
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# LCEL 체인 구성 (JsonOutputParser 사용)
chain = step_prompt | llm | parser

# 질문
text = """
양자 컴퓨팅은 양자역학의 원리를 바탕으로 데이터를 처리하는 새로운 형태의 계산 방식이다.
기존의 고전적 컴퓨터는 0과 1로 이루어진 이진법(bit)을 사용하여 데이터를 처리하지만,
양자 컴퓨터는 양자 비트(큐비트, qubit)를 사용하여 훨씬 더 복잡하고 빠른 계산을 수행할 수 있다.

큐비트는 동시에 0과 1의 상태를 가질 수 있는 양자 중첩(superposition) 상태를 활용하며,
이를 통해 병렬 계산과 같은 고급 기능이 가능하다.
"""

# 체인 실행
output = chain.invoke({"text": text})

# 결과 출력
pprint(output)
print(type(output))
print("-" * 100)

# JsonOutputParser는 딕셔너리를 반환하므로 키로 접근
print(f"요약: {output['summary']}")
print(f"키워드: {output['keywords']}")
print(f"감정: {output['sentiment']}")

{'keywords': ['양자 컴퓨팅', '큐비트', '중첩'],
 'sentiment': '중립',
 'summary': '양자 컴퓨팅은 양자역학 원리를 이용해 큐비트를 통해 기존 컴퓨터보다 더 빠르고 복잡한 계산을 수행하는 새로운 계산 '
            '방식이다.'}
<class 'dict'>
----------------------------------------------------------------------------------------------------
요약: 양자 컴퓨팅은 양자역학 원리를 이용해 큐비트를 통해 기존 컴퓨터보다 더 빠르고 복잡한 계산을 수행하는 새로운 계산 방식이다.
키워드: ['양자 컴퓨팅', '큐비트', '중첩']
감정: 중립


In [ ]:
# pydantic 사용
from typing import List, Literal
from pydantic import BaseModel, Field

class AnalysisResult(BaseModel):
    """텍스트 분석 결과를 담는 스키마"""
    
    summary: str = Field(
        ...,  # ... 은 required 필드를 의미 (필수 입력, None 허용 안함)
        description="텍스트의 핵심 내용 요약"
    )
    
    keywords: List[str] = Field(
        ...,
        description="텍스트에서 추출한 주요 키워드"
    )
    
    sentiment: Literal["긍정", "부정", "중립"] = Field(
        ...,
        description="텍스트의 전반적인 감정 분석 결과"
    )

structured_llm = llm.with_structured_output(AnalysisResult)

# LCEL
chain = step_prompt | structured_llm

# 질문
output = chain.invoke({"text": text})
print(output)
print(type(output))
print("-"*100)
print(output.summary)
print(output.keywords)
print(output.sentiment)

# [실습 프로젝트]

### 맞춤형 학습 도우미 챗봇 만들기

- 특정 주제에 대한 학습을 돕는 챗봇을 만들어봅니다. 앞서 배운 프롬프트 유형들을 조합하여 활용합니다.

- (1) 퀴즈 문제 예시를 보고, (2) 개념 설명 체인을 완성합니다. 

`(1) 퀴즈 문제 출제 (예시)`

In [49]:
from typing import List
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 퀴즈 문제 스키마
class QuizQuestion(BaseModel):
    """퀴즈 문제 스키마"""
    question: str = Field(..., description="퀴즈 문제")
    options: List[str] = Field(..., description="보기 (4개)")
    correct_answer: int = Field(..., description="정답 번호 (1-4)")
    explanation: str = Field(..., description="정답 설명")


# 퀴즈 생성을 위한 구조화된 출력 프롬프트
quiz_prompt = PromptTemplate(
    template="""다음 주제에 대한 퀴즈 문제를 만들어주세요:
    
주제: {topic}
난이도(상/중/하): {difficulty}

다음 조건을 만족하는 퀴즈를 생성해주세요:
1. 문제는 명확하고 이해하기 쉽게
2. 4개의 보기 제공
3. 정답과 오답은 비슷한 수준으로
4. 상세한 정답 설명 포함""",
    input_variables=["topic", "difficulty"]
)

# LLM 정의
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0.3)

# 구조화된 출력 파서 설정
structured_llm = llm.with_structured_output(QuizQuestion)

# LCEL 체인 구성
chain = quiz_prompt | structured_llm

# 질문 실행
output = chain.invoke({"topic": "인공지능", "difficulty": "상"})

# 결과 출력
pprint(f"퀴즈 문제: {output.question}")
pprint(f"보기: {output.options}")
pprint(f"정답: {output.correct_answer}")
pprint(f"정답 설명: {output.explanation}")

"퀴즈 문제: 인공지능 분야에서 '강화학습'이란 무엇을 의미하며, 그 주요 구성 요소 중 하나가 아닌 것은 무엇인가?"
("보기: ['에이전트가 환경과 상호작용하며 보상을 최대화하는 학습 방법', '환경, 에이전트, 보상 함수, 정책', '데이터 레이블을 "
 "기반으로 한 지도 학습 방식', '에이전트가 행동을 선택하는 기준인 정책']")
'정답: 3'
('정답 설명: 강화학습은 에이전트가 환경과 상호작용하며 보상을 최대화하는 방향으로 학습하는 방법입니다. 주요 구성 요소로는 환경, '
 "에이전트, 보상 함수, 그리고 에이전트가 행동을 결정하는 정책이 포함됩니다. '데이터 레이블을 기반으로 한 지도 학습 방식'은 강화학습이 "
 '아닌 지도학습의 정의이므로 정답이 아닙니다.')


`(2) 개념 설명 체인 (문제)`

In [51]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List
from pprint import pprint

# LLM 설정 (기존과 동일)
llm = ChatOpenAI(
    model='gpt-4.1-mini',
    temperature=0.3,
    top_p=0.9,
)

# 개념 설명을 위한 스키마 (필드 타입 수정)
class ConceptExplanation(BaseModel):
    """개념 설명 스키마"""
    topic: str = Field(..., description="주제 이름")
    explanation: str = Field(..., description="주제에 대한 자세한 설명")
    examples: List[str] = Field(..., description="실제 적용 예시들")
    related_concepts: List[str] = Field(..., description="관련된 다른 개념들")

# JsonOutputParser 설정 (Pydantic 모델 지정)
parser = JsonOutputParser(pydantic_object=ConceptExplanation)

# 개념 설명을 위한 지시형 프롬프트 (템플릿 문자열 추가)
concept_prompt = PromptTemplate(
    template="""다음 주제에 대해 {difficulty} 수준으로 설명해주세요:

주제: {topic}

{format_instructions}
""",
    input_variables=["topic", "difficulty"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 구조화된 출력 파서 설정 (JsonOutputParser 사용)
structured_llm = None  # 사용하지 않음

# LCEL 체인 구성 (JsonOutputParser 사용)
chain = concept_prompt | llm | parser

# 질문 실행
output = chain.invoke({"topic": "인공지능", "difficulty": "하"})

# 결과 출력 (딕셔너리 키로 접근)
pprint(f"주제: {output['topic']}")
pprint(f"설명: {output['explanation']}")
pprint(f"예시: {output['examples']}")
pprint(f"관련 개념: {output['related_concepts']}")

'주제: 인공지능'
('설명: 인공지능은 컴퓨터가 사람처럼 생각하고 배우며 문제를 해결할 수 있도록 만드는 기술입니다. 즉, 컴퓨터가 스스로 학습하고 판단할 수 '
 '있게 하는 것입니다. 인공지능은 우리가 일상생활에서 사용하는 여러 가지 기계와 프로그램에 적용되어 편리함을 줍니다.')
("예시: ['스마트폰 음성인식 비서(예: 시리, 빅스비)', '자동으로 움직이는 자율주행 자동차', '인터넷에서 내가 좋아할 만한 영상을 "
 "추천해 주는 서비스', '사진에서 사람 얼굴을 인식하는 기능']")
"관련 개념: ['기계학습', '딥러닝', '빅데이터', '로봇공학']"
